<a href="https://colab.research.google.com/github/beotavalo/agent-crews-genai/blob/main/Agents_and_Multiagents_GenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Agents with Langchain

In [38]:
from IPython.display import clear_output

In [39]:
!pip install langchain openai python-dotenv

In [40]:
!pip install langchain-community

In [41]:
!pip install -U duckduckgo-search

In [42]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.schema import AgentAction, AgentFinish
from langchain.memory import ConversationBufferMemory
from langchain.tools import DuckDuckGoSearchRun
from typing import List, Union
import re
import os
from dotenv import load_dotenv

In [43]:
"""import os
with open("//content/Projects_OpenAILangchain.txt") as archivo:
  apikey = archivo.read()
os.environ["OPENAI_API_KEY"] = apikey"""

'import os\nwith open("//content/Projects_OpenAILangchain.txt") as archivo:\n  apikey = archivo.read()\nos.environ["OPENAI_API_KEY"] = apikey'

In [44]:
# Read OPENAI_API_KEY form colab secrets

from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')


In [45]:
search = DuckDuckGoSearchRun()

tools = [
    Tool(
        name="Búsqueda",
        func=search.run,
        description="Útil para cuando necesitas buscar información actualizada en internet."
    ),
    Tool(
        name="Escalamiento",
        func=lambda x: "Esta consulta ha sido escalada a un agente humano.",
        description="Usa esta herramienta cuando la consulta sea demasiado compleja o requiera intervención humana."
    )
]

In [46]:
template = """Eres un agente de atención al cliente. Usa las siguientes herramientas para responder las consultas de los clientes:

{tools}

Usa el siguiente formato:

Pregunta: la pregunta del cliente
Pensamiento: siempre piensa paso a paso qué hacer
Acción: la acción a tomar, debe ser una de [{tool_names}]
Entrada de acción: la entrada a la acción
Observación: el resultado de la acción
... (este Pensamiento/Acción/Entrada de acción/Observación puede repetirse N veces)
Pensamiento: Ahora sé la respuesta final
Respuesta final: la respuesta final a la pregunta del cliente

Pregunta: {input}
{agent_scratchpad}"""

class CustomPromptTemplate(StringPromptTemplate):
    template: str
    tools: List[Tool]

    def format(self, **kwargs) -> str:
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservación: {observation}\nPensamiento: "
        kwargs["agent_scratchpad"] = thoughts
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    input_variables=["input", "intermediate_steps"]
)

In [47]:
class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        if "Respuesta final:" in llm_output:
            return AgentFinish(
                return_values={"output": llm_output.split("Respuesta final:")[-1].strip()},
                log=llm_output,
            )

        regex = r"Acción: (\w+)[\n]*Entrada de acción: (.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"No se pudo parsear la salida del LLM: {llm_output}")
        action = match.group(1).strip()
        action_input = match.group(2)
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

output_parser = CustomOutputParser()

In [48]:
llm = OpenAI(temperature=0)

llm_chain = LLMChain(llm=llm, prompt=prompt)

agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservación:"],
    allowed_tools=[tool.name for tool in tools]
)

memory = ConversationBufferMemory(memory_key="chat_history")

agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, memory=memory)

In [49]:
def manejar_consulta(consulta):
    respuesta = agent_executor.run(consulta)
    if "escalada a un agente humano" in respuesta:
        print("La consulta ha sido escalada a un agente humano.")
    else:
        print("Respuesta automática:", respuesta)

In [50]:
manejar_consulta("¿Que sabes sobre Venta Consultiva?")



> Entering new AgentExecutor chain...
Pensamiento: Primero, debo buscar información actualizada sobre Venta Consultiva en internet.
Acción: Búsqueda
Entrada de acción: Venta Consultiva

Observación:Aprende qué es la venta consultiva, un tipo de ventas donde el agente trabaja como un consultor o asesor individual para el cliente. Descubre sus características, ventajas y cómo aplicarla con ejemplos y recursos de Zendesk. Aprende qué es la venta consultiva, una técnica de ventas que se centra en entender y resolver las necesidades y problemas de los clientes. Descubre sus beneficios, los pasos para implementarla y algunos ejemplos prácticos. La venta consultiva es un enfoque de ventas que favorece la construcción de relaciones y el diálogo abierto para satisfacer adecuadamente las necesidades de un cliente potencial. Al generar confianza rápidamente, un enfoque de venta consultivo puede ayudar al cliente a cumplir mejor sus expectativas y al vendedor a alcanzar sus objetivos con mayor e

# Multiagents with CrewAI

Building a Web Search Tool with CrewAI

In [51]:
!pip install crewai-tools crewai

**Step 1: Scraping a website**

We first import the necessary libraries and initialize the ScrapeWebsiteTool. This tool is used to extract content from a website. In this case, it’s set to scrape content from this [webpage](https://www.nobelprize.org/contact):

In [65]:
from crewai_tools import ScrapeWebsiteTool, FileWriterTool, TXTSearchTool
import requests

# Initialize the tool, potentially passing the session
tool = ScrapeWebsiteTool(website_url='https://www.nobelprize.org/contact')

# Extract the text
text = tool.run()
print(text)

Using Tool: Read website content
The official website of the Nobel Prize - NobelPrize.org
Skip to content 
Close the search form
A button that allows you to close the search form if needed
Close the search form
A button that allows you to close the search form if needed
Close Mobile Menu
Toggle Mobile Menu 
Nobel Prizes & laureates
All Nobel Prizes
All Nobel Prizes 2024
Physics prize
Chemistry prize
Medicine prize
Literature prize
Peace prize
Prize in economic sciences
About
About the prize
Alfred Nobel
Award ceremonies
Nomination
Organisation
Stories
Educational
Events & museums
Close the search form
A button that allows you to close the search form if needed
Close the search form
A button that allows you to close the search form if needed
Header Search 
Submit a search term
Allows users to submit a search term
Contact us 
Name* Email*
Contact Purpose*Contact PurposeCopyrightTechnicalLicensing - footage & publishing rightsPressGeneral questionsSubject* Message*CAPTCHA
Contact the Nobe

**Step2: Write the extracted text to a file**

We now use the FileWriterTool to save the extracted text into a file named ai.txt.

In [66]:
# Initialize the tool
file_writer_tool = FileWriterTool()

# Write content to a file in a specified directory
result = file_writer_tool._run(filename='ai.txt', content = text, directory = '', overwrite=True)
print(result)

Content successfully written to ai.txt


**Step 3: Set up the text search tool**

We set up another tool to search the contents of the file ai.txt that we just saved. We also set the environment variable for the OpenAI API key.

In [61]:
"""import os
from crewai_tools import TXTSearchTool

os.environ['OPENAI_API_KEY'] = 'API-KEY'

# Initialize the tool with a specific text file, so the agent can search within the given text file's content
tool = TXTSearchTool(txt='ai.txt')"""

"import os\nfrom crewai_tools import TXTSearchTool\n\nos.environ['OPENAI_API_KEY'] = 'API-KEY'\n\n# Initialize the tool with a specific text file, so the agent can search within the given text file's content\ntool = TXTSearchTool(txt='ai.txt')"

**Step 4: Create an agent for the task and execute it**

We create a data analyst agent with the role of an educator. This agent's task is to answer the question "What is Natural Language Processing?" based on the text we searched in the file.

In [67]:
from crewai import Agent, Task, Crew

context = tool.run()

data_analyst = Agent(
    role='Sales assistant',
    goal=f'Based on the context provided, answer the question - What is the company contacts? Context - {context}',
    backstory='You are a data mining for leads prospection expert',
    verbose=True,
    allow_delegation=False,
    tools=[tool]
)

test_task = Task(
    description="Understand the topic and give the correct response",
    tools=[tool],
    agent=data_analyst,
    expected_output='Give a correct response in JSON data format'
)

crew = Crew(
    agents=[data_analyst],
    tasks=[test_task]
)

output = crew.kickoff()

Using Tool: Read website content


# Agent: Sales assistant
## Task: Understand the topic and give the correct response


# Agent: Sales assistant
## Using tool: Read website content
## Tool Input: 
"{\"name\": \"company contacts\", \"description\": \"\", \"args_schema\": {}, \"return_direct\": false, \"verbose\": false}"
## Tool Output: 
Error: the Action Input is not a valid key, value dictionary.


# Agent: Sales assistant
## Using tool: Read website content
## Tool Input: 
"{\"name\": \"company contacts\", \"description\": \"\", \"args_schema\": {}, \"return_direct\": false, \"verbose\": false}"
## Tool Output: 
Error: the Action Input is not a valid key, value dictionary.


# Agent: Sales assistant
## Using tool: Read website content
## Tool Input: 
"{\"name\": \"company contacts\", \"description\": \"\", \"args_schema\": {}, \"return_direct\": false, \"verbose\": false}"
## Tool Output: 
Error: the Action Input is not a valid key, value dictionary.


# Agent: Sales assistant
## Using tool: Read website content
## 

References:

1. [CrewAI: A Guide With Examples of Multi AI Agent Systems](https://www.datacamp.com/tutorial/crew-ai)
2. [Langchain Tools and Agents use cases with examples](https://telestreak.com/tech/langchain-tools-agents-with-examples/)